In [ ]:
import numpy as np 
import pandas as pd 
import math
import os
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import hashlib
import tensorflow as tf, tensorflow.keras.backend as K
from tensorflow.keras import models
from kaggle_datasets import KaggleDatasets
from tensorflow.keras.layers import GlobalAveragePooling2D,Dense
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tensorflow.keras.layers import Layer
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.callbacks import ModelCheckpoint

!pip install -q efficientnet
import efficientnet.tfkeras as efn

In [ ]:
main_path = '../input/plant-pathology-2020-fgvc7/'
train_df = pd.read_csv(os.path.join(main_path,'train.csv'))
test_df = pd.read_csv(os.path.join(main_path,'test.csv'))
sample = pd.read_csv(os.path.join(main_path,'sample_submission.csv'))

train_df = train_df.drop([1703,1505,379,1173],axis=0).reset_index()

In [ ]:
DIMS = (768,768,3)
EPOCHS = 40
SEED = 0
#FOLDS = 5
TTA = 5
#MULTIPLE_OVERSAMPLE_NUM = 400

In [ ]:
print("Tensorflow version " + tf.__version__)
AUTO = tf.data.experimental.AUTOTUNE

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)
BATCH_SIZE = 8 * strategy.num_replicas_in_sync
print('BATCH_SIZE = {}'.format(BATCH_SIZE))

gcs_path = KaggleDatasets().get_gcs_path('plant-pathology-2020-fgvc7')

In [ ]:
def format_path(st):
    return gcs_path + '/images/' + st + '.jpg'

paths = train_df.image_id.apply(format_path).values
labels = train_df.loc[:, 'healthy':].values

train_paths, val_paths, train_labels, val_labels = train_test_split(paths, labels, test_size=0.15, random_state=SEED)

test_paths = test_df.image_id.apply(format_path).values


In [ ]:

def get_mat(rotation, shear, height_zoom, width_zoom ):
    
    ############## author : chris deotte ##################

    rotation = math.pi * rotation / 180.
    shear = math.pi * shear / 180.
    
    c1 = tf.math.cos(rotation)
    s1 = tf.math.sin(rotation)
    one = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    rotation_matrix = tf.reshape( tf.concat([c1,s1,zero, -s1,c1,zero, zero,zero,one],axis=0),[3,3] )
        
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)
    shear_matrix = tf.reshape( tf.concat([one,s2,zero, zero,c2,zero, zero,zero,one],axis=0),[3,3] )    

    zoom_matrix = tf.reshape( tf.concat([height_zoom,zero,zero, zero,width_zoom,zero, zero,zero,one],axis=0),[3,3] )
       
    return K.dot(K.dot(rotation_matrix, shear_matrix), zoom_matrix)


In [ ]:
def decode_images(filename,label=None,image_size=(DIMS[0],DIMS[1])):
    bits = tf.io.read_file(filename)
    img = tf.image.decode_jpeg(bits,channels=3)
    img = tf.cast(img,tf.float32)/255.0
    img_mean = tf.keras.backend.mean(img)
    img_std = tf.keras.backend.std(img)
    img -= img_mean
    img /= img_std
    img = tf.image.resize(img, (DIMS[0], DIMS[1]) )
    
    if label is None:
        return img
    else:
        return img, label
       

def augment_data(img,label=None):
    
    img = tf.image.random_brightness(img, 0.2,seed = SEED)
    img = tf.image.random_contrast(img,0.8,1.2,seed = SEED)
    #img = tf.image.rot90(img, k = int(np.random.rand()*100) % 4)
    img = tf.image.random_flip_up_down(img, seed = SEED)
    img = tf.image.random_flip_left_right(img, seed = SEED)
    #img = tf.image.resize(img, (DIMS[0], DIMS[1]) )
    
    if label is None:
        return img
    else:
        return img,label
    

def transform(image,label=None):
    DIM = DIMS[0]
    XDIM = DIM % 2 
    
    rot = 90 * tf.random.uniform(shape=[1])
    shr = 20 * tf.random.uniform(shape=[1]) 
    zoom_factor = tf.random.uniform(shape=[1],minval=0.9, maxval=1.1, seed = SEED)
    h_zoom, w_zoom = zoom_factor, zoom_factor

    m = get_mat(rot,shr,h_zoom,w_zoom) 

    x = tf.repeat( tf.range(DIM//2,-DIM//2,-1), DIM )
    y = tf.tile( tf.range(-DIM//2,DIM//2),[DIM] )
    z = tf.ones([DIM*DIM],dtype='int32')
    idx = tf.stack( [x,y,z] )

    idx2 = K.dot(m,tf.cast(idx,dtype='float32'))
    idx2 = K.cast(idx2,dtype='int32')
    idx2 = K.clip(idx2,-DIM//2+XDIM+1,DIM//2)
              
    idx3 = tf.stack( [DIM//2-idx2[0,], DIM//2-1+idx2[1,]] )
    d = tf.gather_nd(image,tf.transpose(idx3))
        
    if label is None:
        return tf.reshape(d,[DIM,DIM,3])
    else:
        return tf.reshape(d,[DIM,DIM,3]),label



In [ ]:
def get_training_dataset(train_paths,train_labels):
    train_dataset = (tf.data.Dataset
                    .from_tensor_slices((train_paths, train_labels))
                    .map(decode_images, num_parallel_calls=AUTO)
                    .map(augment_data, num_parallel_calls=AUTO)
                    .shuffle(512)
                    #.map(transform, num_parallel_calls=AUTO)
                    .batch(BATCH_SIZE)
                    .prefetch(AUTO)
                    )
    return train_dataset

def get_validation_dataset(valid_paths,valid_labels):
    valid_dataset = (tf.data.Dataset
                    .from_tensor_slices((valid_paths, valid_labels))
                    .map(decode_images, num_parallel_calls=AUTO)
                    .map(augment_data, num_parallel_calls=AUTO)
                    .shuffle(512)
                    #.map(transform, num_parallel_calls=AUTO)
                    .batch(BATCH_SIZE)
                    .prefetch(AUTO)
                    )
    return valid_dataset

def get_test_dataset(test_paths):
    test_dataset = (tf.data.Dataset
                    .from_tensor_slices((test_paths))
                    .map(decode_images, num_parallel_calls=AUTO)
                    .map(augment_data, num_parallel_calls=AUTO)
                    #.map(transform, num_parallel_calls=AUTO)
                    .batch(BATCH_SIZE)
                    )
    return test_dataset

In [ ]:

def get_model():
    with strategy.scope():
        model = tf.keras.Sequential([
            efn.EfficientNetB7(input_shape=DIMS,
                               weights='noisy-student',
                               include_top=None),
            GlobalAveragePooling2D(),
            Dense(4, activation='softmax')
        ])
        
        model.compile(
            optimizer='adam',
            loss = 'categorical_crossentropy',
            metrics=[ tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
                      tf.keras.metrics.AUC(name='auc')
                    ]
            )

        return model


In [ ]:
LR_START = 0.00001
LR_MAX = 0.0001 * strategy.num_replicas_in_sync
LR_MIN = 0.000001
LR_RAMPUP_EPOCHS = 10
LR_SUSTAIN_EPOCHS = 5
LR_EXP_DECAY = .8

def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr
    
lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=True)

rng = [i for i in range(EPOCHS)]
y = [lrfn(x) for x in rng]
plt.plot(rng, y)
print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(y[0], max(y), y[-1]))

ch_p = ModelCheckpoint(filepath="model_ef.h5", monitor="val_loss", 
                       save_best_only=True, mode='min',
                       save_weights_only=True, verbose=1)


In [ ]:
histories = []
STEPS_PER_EPOCH = labels.shape[0] // BATCH_SIZE
total_predictions = []


for i in range(FOLDS):
    print('-'*30)
    print('FOLD NUMBER : {}'.format(i+1))
    print('-'*30)
    
    tf.tpu.experimental.initialize_tpu_system(tpu)
    model = get_model()
    
    history = model.fit(get_training_dataset(paths,labels), 
                        epochs = EPOCHS, 
                        steps_per_epoch = STEPS_PER_EPOCH,
                        callbacks = [lr_callback, ch_p], 
                        verbose = 1)
    histories.append(history)
    
    for i in range(TTA):
        preds = model.predict(get_test_dataset(test_paths), verbose=1)
        total_predictions.append(preds)
    del model


In [ ]:
"""
STEPS_PER_EPOCH = train_labels.shape[0] // BATCH_SIZE
tf.tpu.experimental.initialize_tpu_system(tpu)
model = get_model()
    
history = model.fit(get_training_dataset(train_paths, train_labels), 
                    epochs = EPOCHS, 
                    steps_per_epoch = STEPS_PER_EPOCH,
                    callbacks = [lr_callback, ch_p],
                    #validation_data = get_validation_dataset(val_paths,val_labels),
                    verbose = 1)
"""

In [ ]:
def display_training_curves(training, validation, title, subplot):
    """
    Source: https://www.kaggle.com/mgornergoogle/getting-started-with-100-flowers-on-tpu
    """
    if subplot%10==1: # set up the subplots on the first call
        plt.subplots(figsize=(10,10), facecolor='#F0F0F0')
        plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.set_facecolor('#F8F8F8')
    ax.plot(training)
    ax.plot(validation)
    ax.set_title('model '+ title)
    ax.set_ylabel(title)
    ax.set_xlabel('epoch')
    ax.legend(['train', 'valid.'])

In [ ]:
display_training_curves(
    history.history['loss'], 
    history.history['val_loss'], 
    'loss', 211)
display_training_curves(
    history.history['categorical_accuracy'], 
    history.history['val_categorical_accuracy'], 
    'accuracy', 212)

In [ ]:
total_predictions =[]
model.load_weights('model_ef.h5')
print('done')
for i in range(1):
    preds = model.predict(get_test_dataset(test_paths), verbose=1)
    total_predictions.append(preds)

In [ ]:
prob_sum = 0
for prob in total_predictions:
    prob_sum += prob
prob_avg = prob_sum/(1)

sample.loc[:, 'healthy':] = prob_avg
sample.to_csv('Enetb7_size_600*900.csv', index=False)
sample.head(30)

In [ ]:
predictions=[]
for i in range(10):
    preds = model.predict(get_test_dataset(test_paths), verbose = 1)
    predictions.append(preds)


final_preds=0
for preds in predictions:
    final_preds += preds
final_preds /= 10

In [ ]:
sample.loc[:, 'healthy':] = final_preds
sample.to_csv('submission.csv', index=False)
sample.head()

In [ ]:
display_training_curves(
    histories[0].history['loss'], 
    histories[0].history['val_loss'], 
    'loss', 211)
display_training_curves(
    histories[0].history['categorical_accuracy'], 
    histories[0].history['val_categorical_accuracy'], 
    'accuracy', 212)


In [ ]:
display_training_curves(
    histories[1].history['loss'], 
    histories[1].history['val_loss'], 
    'loss', 211)
display_training_curves(
    histories[1].history['categorical_accuracy'], 
    histories[1].history['val_categorical_accuracy'], 
    'accuracy', 212)

In [ ]:
display_training_curves(
    histories[2].history['loss'], 
    histories[2].history['val_loss'], 
    'loss', 211)
display_training_curves(
    histories[2].history['categorical_accuracy'], 
    histories[2].history['val_categorical_accuracy'], 
    'accuracy', 212)